In [2]:
import os
from pathlib2 import Path
import re
import itertools
import copy
import numpy as np
from IPython.display import display_html 
import random


import pyarrow.feather as feather
import pandas as pd
import xml.etree.ElementTree as ET
from dython.nominal import associations
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import (GradientBoostingRegressor, GradientBoostingClassifier)
import miceforest as mf


## 1. Exploratory Data Analysis

This analysis is primarily focused on Categroy 909 to get an understanding of the dataset and device an algorithmic approach for missing data. Learnings from here will be expanded to other categories, however not all as some work might only be applicable to this category.

In [3]:
category_data = feather.read_feather(Path('IceCat_Cat_2833_feather/frame_IceCat_Category_909.feather'))
display(category_data.tail())
display(category_data.info())

,level_0,index,id,name,category_id,category_label,Operating temperature (T-T).1112,Operating temperature (T-T).1112.unit,Maximum data transfer rate.1165,Maximum data transfer rate.1165.unit,...,Compatibility.890,Compatibility.890.unit,Interface type.990,Interface type.990.unit,Firewall security.1612,Firewall security.1612.unit,Works with the Google Assistant.36516,Works with the Google Assistant.36516.unit,RTS/CTS threshold.22398,RTS/CTS threshold.22398.unit
2232,2232,0,IceCat_Prod_16048998,Aironet 2602E,IceCat_Category_909,Wireless Access Points,-20 - 55,°C,450.0,Mbit/s,...,None,None,None,None,None,None,<NA>,None,<NA>,None
2233,2233,0,IceCat_Prod_31985083,Aironet 3700i,IceCat_Category_909,Wireless Access Points,0 - 40,°C,1300.0,Mbit/s,...,None,None,None,None,None,None,<NA>,None,<NA>,None
2234,2234,0,IceCat_Prod_33721921,Aironet 3600e,IceCat_Category_909,Wireless Access Points,-20 - 55,°C,1000.0,Mbit/s,...,None,None,None,None,None,None,<NA>,None,<NA>,None
2235,2235,0,IceCat_Prod_11147703,NWA5550-N,IceCat_Category_909,Wireless Access Points,-40 - 60,°C,300.0,Mbit/s,...,None,None,None,None,None,None,<NA>,None,<NA>,None
2236,2236,0,IceCat_Prod_36197901,Aironet 1562E,IceCat_Category_909,Wireless Access Points,-40 - 65,°C,1300.0,Mbit/s,...,None,None,None,None,None,None,<NA>,None,<NA>,None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2237 entries, 0 to 2236
Columns: 540 entries, level_0 to RTS/CTS threshold.22398.unit
dtypes: boolean(88), float64(69), int64(2), object(381)
memory usage: 8.1+ MB


None

###  1.1 Data Pruning

The dataset has several columns that have missing data. A reasonable threshold is therefore necessary to select columns that have a substantially complete set of values. Given that 100% would result in only product reference columns, 80% was selected which resulted in 29 columns:

In [4]:
pct_complete_threshold = .8
pct_complete = (len(category_data) - category_data.isnull().sum()) / len(category_data)
missing_vals = pd.DataFrame({'col': category_data.columns,
                             'pct_complete': pct_complete})
num_cols_above_null_threshold = missing_vals[missing_vals['pct_complete'] >= (pct_complete_threshold)].count()

cols_of_interest = missing_vals[missing_vals['pct_complete'] >= pct_complete_threshold].iloc[:,0]

print(f"Columns above {pct_complete_threshold:.0%} percent complete threshold ({num_cols_above_null_threshold[0]}):")
display(cols_of_interest.values)

Columns above 80% percent complete threshold (29):


array(['level_0', 'index', 'id', 'name', 'category_id', 'category_label',
       'Operating temperature (T-T).1112',
       'Operating temperature (T-T).1112.unit',
       'Maximum data transfer rate.1165',
       'Maximum data transfer rate.1165.unit', 'Height.1464',
       'Height.1464.unit', 'Width.1649', 'Width.1649.unit', 'Depth.1650',
       'Depth.1650.unit', 'Product colour.1766',
       'Networking standards.1802', '2.4 GHz.20806', '5 GHz.20807',
       'Ethernet LAN (RJ-45) ports.2312', 'Ethernet LAN data rates.3768',
       'Security algorithms.454', 'Storage temperature (T-T).757',
       'Storage temperature (T-T).757.unit', 'Weight.94',
       'Weight.94.unit', 'Operating relative humidity (H-H).703',
       'Operating relative humidity (H-H).703.unit'], dtype=object)

Columns can be broken into two buckets:
- *Unit based columns*: these will be filled based on allowable values based on *CategoryFeatureslist.xml* file as filled values are consistent within columns
- *Non-unit based columns*: these will be further investigated for possible inclusion in imputation analysis

In [5]:
cols_units = [str for str in cols_of_interest.values if ".unit" in str]
cols_sys = ["level_0", "index", "id", "category_id", "category_label"]
cols_non_units = set(cols_of_interest.values) - set(cols_units) - set(cols_sys)
print(f"Unit based columns ({len(cols_units)}):")
display(cols_units)
print(f"System reference columns ({len(cols_sys)}):")
display(cols_sys)
print(f"Non-unit based columns ({len(cols_non_units)}):")
display(list(cols_non_units))

Unit based columns (8):


['Operating temperature (T-T).1112.unit',
 'Maximum data transfer rate.1165.unit',
 'Height.1464.unit',
 'Width.1649.unit',
 'Depth.1650.unit',
 'Storage temperature (T-T).757.unit',
 'Weight.94.unit',
 'Operating relative humidity (H-H).703.unit']

System reference columns (5):


['level_0', 'index', 'id', 'category_id', 'category_label']

Non-unit based columns (16):


['name',
 'Networking standards.1802',
 'Ethernet LAN data rates.3768',
 'Security algorithms.454',
 'Operating relative humidity (H-H).703',
 'Maximum data transfer rate.1165',
 '2.4 GHz.20806',
 'Weight.94',
 'Width.1649',
 'Storage temperature (T-T).757',
 'Depth.1650',
 '5 GHz.20807',
 'Operating temperature (T-T).1112',
 'Height.1464',
 'Product colour.1766',
 'Ethernet LAN (RJ-45) ports.2312']

When we look at the uniqueness of each column, two columns (`2.4 GHz.20806` & `5 GHz.20807`) stand out as having only one value throughout each column (other than null) and are therefore removed:

In [6]:
uniqueness = category_data[list(cols_non_units)].apply(pd.Series.nunique).sort_values()
print("Variation within each column:")
display(uniqueness)
low_variation_cols = list(uniqueness[uniqueness == 1].index)
print(f'Columns with no variation: {low_variation_cols}')
cols = [col for col in list(cols_non_units) if col not in low_variation_cols]
print(f'\nColumns to be considered:')
display(cols)

Variation within each column:


2.4 GHz.20806                              1
5 GHz.20807                                1
Ethernet LAN (RJ-45) ports.2312            8
Product colour.1766                       12
Ethernet LAN data rates.3768              18
Operating relative humidity (H-H).703     27
Storage temperature (T-T).757             31
Operating temperature (T-T).1112          33
Maximum data transfer rate.1165           62
Width.1649                               186
Height.1464                              195
Depth.1650                               214
Weight.94                                233
Networking standards.1802                365
Security algorithms.454                  422
name                                     787
dtype: int64

Columns with no variation: ['2.4 GHz.20806', '5 GHz.20807']

Columns to be considered:


['name',
 'Networking standards.1802',
 'Ethernet LAN data rates.3768',
 'Security algorithms.454',
 'Operating relative humidity (H-H).703',
 'Maximum data transfer rate.1165',
 'Weight.94',
 'Width.1649',
 'Storage temperature (T-T).757',
 'Depth.1650',
 'Operating temperature (T-T).1112',
 'Height.1464',
 'Product colour.1766',
 'Ethernet LAN (RJ-45) ports.2312']

Below is a preview of the data thus far after dropping observations w/ null values (maybe consider including in next iteration?). Note that there are columns with several attributes within a cell. For example, for 1st observation below the `Networking standards.1802` attribute includes several values separated by a comma: `IEEE 802.11a,IEEE 802.11b,IEEE 802.11g,IEEE 802.11n,IEEE 802.3af`. To appropriately capture the attributes for each observation, dummy variables will need to be created. 

In [7]:
pruned_data = category_data[cols].dropna()
pruned_data.head()

,name,Networking standards.1802,Ethernet LAN data rates.3768,Security algorithms.454,Operating relative humidity (H-H).703,Maximum data transfer rate.1165,Weight.94,Width.1649,Storage temperature (T-T).757,Depth.1650,Operating temperature (T-T).1112,Height.1464,Product colour.1766,Ethernet LAN (RJ-45) ports.2312
6,Aironet 702i,"IEEE 802.11a,IEEE 802.11b,IEEE 802.11g,IEEE 80...","10,100,1000","EAP,EAP-SIM,EAP-TLS,EAP-TTLS,PEAP,TKIP,WPA,WPA2",10 - 90,1000.0,480.0,177.6,-30 - 70,177.6,0 - 40,50.4,White,2.0
7,Aironet 2802i,"IEEE 802.11a,IEEE 802.11ac,IEEE 802.11b,IEEE 8...","100,1000","802.1x RADIUS,AES,EAP-FAST,EAP-PEAP,EAP-SIM,EA...",10 - 90,5200.0,1600.0,220.0,-30 - 70,220.4,-20 - 50,55.1,White,2.0
10,Aironet 2802i,"IEEE 802.11a,IEEE 802.11ac,IEEE 802.11b,IEEE 8...","100,1000","802.1x RADIUS,AES,EAP-FAST,EAP-PEAP,EAP-SIM,EA...",10 - 90,5200.0,1600.0,220.0,-30 - 70,220.4,-20 - 50,55.1,White,2.0
12,Aironet 1815w,"IEEE 802.11a,IEEE 802.11ac,IEEE 802.11b,IEEE 8...","10,100,1000","802.1x RADIUS,AES,EAP,EAP-FAST,EAP-PEAP,EAP-SI...",10 - 90,1000.0,280.0,89.0,-30 - 70,140.0,0 - 40,31.5,White,3.0
14,Aironet 2602i,"IEEE 802.11a,IEEE 802.11b,IEEE 802.11d,IEEE 80...","10,100,1000","802.1x RADIUS,AES,EAP-FAST,EAP-PEAP,EAP-SIM,EA...",10 - 90,1000.0,1040.0,221.0,-30 - 70,54.0,0 - 40,221.0,White,1.0


The columns identified that would need dummy variables created are:
- `Product colour.1766`
- `Networking standards.1802`
- `Ethernet LAN data rates.3768 `
- `Security algorithms.454` 

In [8]:
cols_need_dummies = ["Product colour.1766", "Networking standards.1802", "Ethernet LAN data rates.3768", "Security algorithms.454"]
pd.DataFrame({"col": cols_need_dummies, 
              "dummy_var_count": [pruned_data[col].str.get_dummies(",").shape[1] for col in cols_need_dummies]})

,col,dummy_var_count
0,Product colour.1766,6
1,Networking standards.1802,40
2,Ethernet LAN data rates.3768,6
3,Security algorithms.454,48


In [9]:
xgdata = pd.concat([pruned_data, 
                    pruned_data["Product colour.1766"].str.get_dummies(","),
                    pruned_data["Networking standards.1802"].str.get_dummies(","),
                    pruned_data["Ethernet LAN data rates.3768"].str.get_dummies(","),
                    pruned_data["Security algorithms.454"].str.get_dummies(",")],
                    axis=1) \
            .drop(columns=cols_need_dummies)

print(f'The new dataset is now {xgdata.shape[0]:,} by {xgdata.shape[1]}.\n\nBelow is a preview:')

The new dataset is now 1,316 by 110.

Below is a preview:


In [10]:
xgdata.head()

,name,Operating relative humidity (H-H).703,Maximum data transfer rate.1165,Weight.94,Width.1649,Storage temperature (T-T).757,Depth.1650,Operating temperature (T-T).1112,Height.1464,Ethernet LAN (RJ-45) ports.2312,...,WPA2,WPA2-AES,WPA2-CCMP,WPA2-Enterprise,WPA2-PSK,WPA2-TKIP,WPA3,WPA3-Enterprise,WPA3-PSK,WPS
6,Aironet 702i,10 - 90,1000.0,480.0,177.6,-30 - 70,177.6,0 - 40,50.4,2.0,...,1,0,0,0,0,0,0,0,0,0
7,Aironet 2802i,10 - 90,5200.0,1600.0,220.0,-30 - 70,220.4,-20 - 50,55.1,2.0,...,1,0,0,0,0,0,0,0,0,0
10,Aironet 2802i,10 - 90,5200.0,1600.0,220.0,-30 - 70,220.4,-20 - 50,55.1,2.0,...,1,0,0,0,0,0,0,0,0,0
12,Aironet 1815w,10 - 90,1000.0,280.0,89.0,-30 - 70,140.0,0 - 40,31.5,3.0,...,1,0,0,0,0,0,0,0,0,0
14,Aironet 2602i,10 - 90,1000.0,1040.0,221.0,-30 - 70,54.0,0 - 40,221.0,1.0,...,1,0,0,0,0,0,0,0,0,0


Dummy variables will also need to be created on the below categorical columns:

In [11]:
add_col_dummies = list(xgdata.select_dtypes(['object']).columns)
print(add_col_dummies)

['name', 'Operating relative humidity (H-H).703', 'Storage temperature (T-T).757', 'Operating temperature (T-T).1112']


In [12]:
xgdata_final = pd.concat([xgdata, 
                    pd.get_dummies(xgdata[add_col_dummies])],
                    axis=1) \
                 .drop(columns=add_col_dummies)

In [13]:
xgdata.head()

,name,Operating relative humidity (H-H).703,Maximum data transfer rate.1165,Weight.94,Width.1649,Storage temperature (T-T).757,Depth.1650,Operating temperature (T-T).1112,Height.1464,Ethernet LAN (RJ-45) ports.2312,...,WPA2,WPA2-AES,WPA2-CCMP,WPA2-Enterprise,WPA2-PSK,WPA2-TKIP,WPA3,WPA3-Enterprise,WPA3-PSK,WPS
6,Aironet 702i,10 - 90,1000.0,480.0,177.6,-30 - 70,177.6,0 - 40,50.4,2.0,...,1,0,0,0,0,0,0,0,0,0
7,Aironet 2802i,10 - 90,5200.0,1600.0,220.0,-30 - 70,220.4,-20 - 50,55.1,2.0,...,1,0,0,0,0,0,0,0,0,0
10,Aironet 2802i,10 - 90,5200.0,1600.0,220.0,-30 - 70,220.4,-20 - 50,55.1,2.0,...,1,0,0,0,0,0,0,0,0,0
12,Aironet 1815w,10 - 90,1000.0,280.0,89.0,-30 - 70,140.0,0 - 40,31.5,3.0,...,1,0,0,0,0,0,0,0,0,0
14,Aironet 2602i,10 - 90,1000.0,1040.0,221.0,-30 - 70,54.0,0 - 40,221.0,1.0,...,1,0,0,0,0,0,0,0,0,0


In [14]:
xgdata_final.head()

,Maximum data transfer rate.1165,Weight.94,Width.1649,Depth.1650,Height.1464,Ethernet LAN (RJ-45) ports.2312,Black,Bronze,Gold,Grey,...,Operating temperature (T-T).1112_-40 - 60,Operating temperature (T-T).1112_-40 - 65,Operating temperature (T-T).1112_-40 - 70,Operating temperature (T-T).1112_-40 - 85,Operating temperature (T-T).1112_0 - 40,Operating temperature (T-T).1112_0 - 45,Operating temperature (T-T).1112_0 - 50,Operating temperature (T-T).1112_0 - 55,Operating temperature (T-T).1112_0 - 65,Operating temperature (T-T).1112_10 - 90
6,1000.0,480.0,177.6,177.6,50.4,2.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7,5200.0,1600.0,220.0,220.4,55.1,2.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,5200.0,1600.0,220.0,220.4,55.1,2.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1000.0,280.0,89.0,140.0,31.5,3.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
14,1000.0,1040.0,221.0,54.0,221.0,1.0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


The dataset is now ready for further analysis for imputation techniques

## 2. Data Imputation

### 2.1 Data removal

In [15]:
def remove_data(df, p_rm):
    df_copy = copy.deepcopy(df.reset_index())

    for col in df_copy:
        if col != "index":
            # continue
            n_val = df_copy[col].notna().sum()
            n_rm = int(n_val * p_rm)
            idx_ = np.random.choice(df_copy[col].shape[0], n_rm, replace=False)

            df_copy.loc[idx_, col] = np.nan

    return df_copy 


In [16]:
# iter_imp = IterativeImputer().fit_transform(rm_10_pct)
# iter_imp


In [17]:
xgdata

,name,Operating relative humidity (H-H).703,Maximum data transfer rate.1165,Weight.94,Width.1649,Storage temperature (T-T).757,Depth.1650,Operating temperature (T-T).1112,Height.1464,Ethernet LAN (RJ-45) ports.2312,...,WPA2,WPA2-AES,WPA2-CCMP,WPA2-Enterprise,WPA2-PSK,WPA2-TKIP,WPA3,WPA3-Enterprise,WPA3-PSK,WPS
6,Aironet 702i,10 - 90,1000.0,480.0,177.6,-30 - 70,177.6,0 - 40,50.4,2.0,...,1,0,0,0,0,0,0,0,0,0
7,Aironet 2802i,10 - 90,5200.0,1600.0,220.0,-30 - 70,220.4,-20 - 50,55.1,2.0,...,1,0,0,0,0,0,0,0,0,0
10,Aironet 2802i,10 - 90,5200.0,1600.0,220.0,-30 - 70,220.4,-20 - 50,55.1,2.0,...,1,0,0,0,0,0,0,0,0,0
12,Aironet 1815w,10 - 90,1000.0,280.0,89.0,-30 - 70,140.0,0 - 40,31.5,3.0,...,1,0,0,0,0,0,0,0,0,0
14,Aironet 2602i,10 - 90,1000.0,1040.0,221.0,-30 - 70,54.0,0 - 40,221.0,1.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2228,Aironet 3802i,10 - 90,5200.0,2090.0,220.0,-30 - 70,220.0,0 - 40,62.5,2.0,...,1,0,0,0,0,0,0,0,0,0
2230,Aironet 1700,10 - 90,1000.0,1000.0,221.0,-30 - 70,221.0,0 - 40,51.0,3.0,...,1,0,0,0,0,0,0,0,0,0
2231,WAP125,10 - 85,867.0,303.0,123.0,-20 - 70,123.0,0 - 40,31.0,1.0,...,1,0,0,0,0,0,0,0,0,0
2233,Aironet 3700i,10 - 90,1300.0,1130.0,221.0,-30 - 70,221.0,0 - 40,54.0,2.0,...,0,0,0,0,0,0,0,0,0,0


## Mice Forest

Attempting to use MICE Forest doesn't yield good results as there are several instances of rare categories for `Storage temperature (T-T).757`, `Operating relative humidity (H-H).703`,`Operating temperature (T-T).1112`. As such we use the weighted distribution to impute for these columns:

In [18]:
xgdata_mice_forest = xgdata.drop(columns=["name"]) # Name has high cardinality

In [19]:
xgdata_mice_forest_amp = remove_data(xgdata_mice_forest, 0.1)
xgdata_mice_forest_amp[xgdata_mice_forest_amp.select_dtypes(["object"]).columns] = xgdata_mice_forest_amp.select_dtypes(["object"]).apply(lambda x: x.astype("category"))

In [20]:
# kds = mf.ImputationKernel(
#   xgdata_mice_forest_amp,
#   save_all_iterations=True,
#   random_state=1991
# )

# # Run the MICE algorithm for 2 iterations
# kds.mice(2)

# # Return the completed dataset.
# xgdata_complete = kds.complete_data()

Replace NAs within categorical columns with selections based on weighted distribution of values in complete dataset:

In [21]:
def impute_cat_weighted_dist(complete_data, amp_data, cat):
    """
    Returns a list of imputed values based on weighted distribution of categories in complete dataset
    """
    cat_complete_dist = complete_data[cat].value_counts() / len(complete_data)
    cat_amp = amp_data[cat]
    cat_na = cat_amp[cat_amp.isna()]
    return random.choices(population=cat_complete_dist.index, weights=cat_complete_dist.values, k=cat_na.size)

In [22]:
cols_to_impute = ["Storage temperature (T-T).757", "Operating relative humidity (H-H).703","Operating temperature (T-T).1112"]

na_idx_by_col = {}

for col in cols_to_impute:
    col_vals = xgdata_mice_forest_amp[col]
    col_vals_na = col_vals[col_vals.isna()]
    na_idx_by_col[col] = list(col_vals_na.index)
    xgdata_mice_forest_amp[col].loc[list(col_vals_na.index)] = impute_cat_weighted_dist(xgdata_mice_forest, xgdata_mice_forest_amp, col)

In [23]:
cols_accuracy = {}

for col in cols_to_impute:
    xgdata_mice_actual = xgdata_mice_forest[col].reset_index(drop=True).loc[na_idx_by_col[col]]
    xgdata_mice_imputed = xgdata_mice_forest_amp[col].loc[na_idx_by_col[col]]
    acc = sum(1 for x, y in zip(xgdata_mice_actual, xgdata_mice_imputed) if x == y) / float(len(xgdata_mice_actual))
    cols_accuracy[col] = acc

In [24]:
cols_accuracy

{'Storage temperature (T-T).757': 0.6183206106870229,
 'Operating relative humidity (H-H).703': 0.732824427480916,
 'Operating temperature (T-T).1112': 0.4122137404580153}

In [25]:
def get_na_idx_by_col(df: pd.DataFrame) -> dict:
    """Return columns with row index of nan values"""
    na_idx_by_col = {}
    df_copy = df.copy()
    cols_with_na_mask = df_copy.apply(lambda col: col.isnull().values.any())
    cols_with_na = cols_with_na_mask[cols_with_na_mask == True].index
    for col in cols_with_na:
        na_idx_by_col[col] = df_copy[df_copy[col].isnull()].index.tolist()
    return na_idx_by_col


In [26]:
def get_imp_cols_accuracy(actual: pd.DataFrame, removed: pd.DataFrame, imputed: pd.DataFrame, ) -> dict:
    """Return imputation accuracy of each column"""
    act_copy = actual.copy().reset_index()
    imp_idx = get_na_idx_by_col(removed)
    cols_acc = {}
    for col, idx_vals in imp_idx.items():
        act = act_copy[col].iloc[idx_vals]
        imp = imputed[col].iloc[idx_vals]
        cols_acc[col] = sum(1 for a, i in zip(act, imp) if a == i) / float(len(act))
    return cols_acc


In [27]:
kds = mf.ImputationKernel(
  xgdata_mice_forest_amp,
  save_all_iterations=True,
  random_state=1991
)

# # Run the MICE algorithm for 2 iterations
kds.mice(2)

# # Return the completed dataset.
xgdata_complete = kds.complete_data()

c:\Users\BensonIgarabuza\miniconda3\envs\isye6748\Lib\site-packages\miceforest\ImputationKernel.py:369: UserWarning: [Operating relative humidity (H-H).703,Storage temperature (T-T).757,Operating temperature (T-T).1112] have very rare categories, it is a good idea to group these, or set the min_data_in_leaf parameter to preventlightgbm from outputting 0.0 probabilities.
  warn(


In [28]:
# get_na_idx_by_col(xgdata_mice_forest_amp)
# xgdata_mice_forest_amp[xgdata_mice_forest_amp["Maximum data transfer rate.1165"].isnull()]
acc_by_col = get_imp_cols_accuracy(xgdata_mice_forest, xgdata_mice_forest_amp, xgdata_complete)
acc_by_col

{'Maximum data transfer rate.1165': 0.7480916030534351,
 'Weight.94': 0.7022900763358778,
 'Width.1649': 0.6564885496183206,
 'Depth.1650': 0.732824427480916,
 'Height.1464': 0.7251908396946565,
 'Ethernet LAN (RJ-45) ports.2312': 0.9236641221374046,
 'Black': 0.9847328244274809,
 'Bronze': 0.9923664122137404,
 'Gold': 1.0,
 'Grey': 0.9923664122137404,
 'Silver': 1.0,
 'White': 0.9847328244274809,
 'IEEE 802.11a': 0.9923664122137404,
 'IEEE 802.11ac': 0.9770992366412213,
 'IEEE 802.11ad': 1.0,
 'IEEE 802.11ax': 0.9770992366412213,
 'IEEE 802.11az': 0.9923664122137404,
 'IEEE 802.11b': 0.9770992366412213,
 'IEEE 802.11d': 0.9541984732824428,
 'IEEE 802.11e': 0.9923664122137404,
 'IEEE 802.11g': 0.9847328244274809,
 'IEEE 802.11h': 0.9770992366412213,
 'IEEE 802.11i': 0.9389312977099237,
 'IEEE 802.11k': 1.0,
 'IEEE 802.11n': 0.9923664122137404,
 'IEEE 802.11r': 0.9923664122137404,
 'IEEE 802.11u': 0.9923664122137404,
 'IEEE 802.11v': 1.0,
 'IEEE 802.11w': 1.0,
 'IEEE 802.15.4': 0.992366

In [29]:
idx = get_na_idx_by_col(xgdata_mice_forest_amp)["Maximum data transfer rate.1165"]
actual = xgdata_mice_forest.reset_index()["Maximum data transfer rate.1165"].iloc[idx]
imputed = xgdata_complete["Maximum data transfer rate.1165"].iloc[idx]


sum(1 for act, imp in zip(actual, imputed) if act == imp) / float(len(actual))

# sum(1 for x, y in zip(xgdata_mice_actual, xgdata_mice_imputed) if x == y) / float(len(xgdata_mice_actual))

0.7480916030534351

Reconstruction dataset back to original form 

In [30]:
xgdata_complete.filter(regex="IEEE.*")

,IEEE 802.11a,IEEE 802.11ac,IEEE 802.11ad,IEEE 802.11ax,IEEE 802.11az,IEEE 802.11b,IEEE 802.11d,IEEE 802.11e,IEEE 802.11g,IEEE 802.11h,...,IEEE 802.3ad,IEEE 802.3af,IEEE 802.3at,IEEE 802.3au,IEEE 802.3az,IEEE 802.3bz,IEEE 802.3i,IEEE 802.3u,IEEE 802.3x,IEEE 802.3z
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1311,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1312,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1313,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1314,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
def stringify_dummy_list(lst: list) -> list:
    combined = [network_col_keys[idx] for idx, v in enumerate(lst) if v == 1 ]
    return ', '.join(combined)


In [68]:
xg_data_complete_networking = xgdata_complete.filter(regex="IEEE.*").copy()
xg_data_complete_networking.loc[:,"networking keys"] = xg_data_complete_networking.apply(lambda x: x.astype(int).tolist(), axis=1)
network_col_keys = {xg_data_complete_networking.columns.get_loc(col):col for col in xg_data_complete_networking.columns}
xg_data_complete_networking.loc[:,"networking vals"] = xg_data_complete_networking["networking keys"].apply(stringify_dummy_list)

# pruned_data["Networking standards.1802"]

In [147]:
# for each col that needs dummy
    # get unique values (col names)
    # get dictionary of idx of each col
    # create new col w/ concatenated col values
    # stringify

def undummify_cols(orig_data: pd.DataFrame, data_to_undummify: pd.DataFrame, dummified_col_list: list) -> pd.DataFrame:
    df_to_undummify = data_to_undummify.copy()
    dummy_cols_to_drop = []
    for col_name in dummified_col_list:
        dummy_col_names = list(set(','.join(orig_data[col_name].unique().tolist()).split(",")))
        dummy_cols_to_drop.extend(dummy_col_names)
        idx_cols_by_col_name = {col:df_to_undummify.columns.get_loc(col) for col in dummy_col_names}
        sorted_dummy_col_names = sorted(dummy_col_names, key=lambda x: idx_cols_by_col_name[x])
        df_to_undummify.loc[:,f"{col_name}_combined_dummies"] = df_to_undummify[sorted_dummy_col_names].apply(lambda x: x.tolist(), axis=1)  #{df_to_undummify.columns.get_loc(col):col for col in dummy_col_names}
        df_to_undummify.loc[:,f"{col_name}_undummified"] = df_to_undummify.loc[:,f"{col_name}_combined_dummies"].apply(lambda x: ','.join([sorted_dummy_col_names[idx] for idx, v in enumerate(x) if v == 1]))
    result = df_to_undummify[df_to_undummify.columns.drop(list(df_to_undummify.filter(regex="_combined_dummies")))]
    return result.drop(columns=dummy_cols_to_drop)

In [148]:
# l = set(','.join(pruned_data["Networking standards.1802"].unique().tolist()).split(","))
r = undummify_cols(pruned_data, xgdata_mice_forest, cols_need_dummies)
r

['Black', 'White', 'Grey', 'Bronze', 'Gold', 'Silver', 'IEEE 802.11b', 'IEEE 802.11u', 'IEEE 802.11k', 'IEEE 802.15.4', 'IEEE 802.1s', 'IEEE 802.3u', 'IEEE 802.11a', 'IEEE 802.3', 'IEEE 802.3af', 'IEEE 802.11d', 'IEEE 802.11e', 'IEEE 802.11az', 'IEEE 802.3z', 'IEEE 802.1Q', 'IEEE 802.11i', 'IEEE 802.11ac', 'IEEE 802.11h', 'IEEE 802.3bz', 'IEEE 802.3ad', 'IEEE 802.11w', 'IEEE 802.11ax', 'IEEE 802.1af', 'IEEE 802.3i', 'IEEE 802.3ac', 'IEEE 802.11g', 'IEEE 802.3x', 'IEEE 802.11r', 'IEEE 802.3ab', 'IEEE 802.3at', 'IEEE 802.11ad', 'IEEE 802.2x', 'IEEE 802.11n', 'IEEE 802.1ag', 'IEEE 802.11v', 'IEEE 802.3au', 'IEEE 802.1AX', 'IEEE 802.1x', 'IEEE 802.3az', 'IEEE 802.1p', 'IEEE 802.1D', '10000', '10', '5000', '100', '2500', '1000', 'IPSec', 'WEP', 'WPA3-Enterprise', 'WPA', 'WPA-PSK', 'WPA3', 'EAP-TTLS', '64-bit WEP', 'HTTPS', 'WPA2-CCMP', 'EAP', 'TTLS', 'WPA2', 'WPA-TKIP', '152-bit WEP', 'WPA3-PSK', 'SNMP', 'WPA2-PSK', 'WPA2-Enterprise', 'EAP-FAST', 'WPA-EAP', 'SSL/TLS', 'EAP-PEAP', 'AES', 'WP

,Operating relative humidity (H-H).703,Maximum data transfer rate.1165,Weight.94,Width.1649,Storage temperature (T-T).757,Depth.1650,Operating temperature (T-T).1112,Height.1464,Ethernet LAN (RJ-45) ports.2312,Product colour.1766_undummified,Networking standards.1802_undummified,Ethernet LAN data rates.3768_undummified,Security algorithms.454_undummified
6,10 - 90,1000.0,480.0,177.6,-30 - 70,177.6,0 - 40,50.4,2.0,White,"IEEE 802.11a,IEEE 802.11b,IEEE 802.11g,IEEE 80...","10,100,1000","EAP,EAP-SIM,EAP-TLS,EAP-TTLS,PEAP,TKIP,WPA,WPA2"
7,10 - 90,5200.0,1600.0,220.0,-30 - 70,220.4,-20 - 50,55.1,2.0,White,"IEEE 802.11a,IEEE 802.11ac,IEEE 802.11b,IEEE 8...","100,1000","802.1x RADIUS,AES,EAP-FAST,EAP-PEAP,EAP-SIM,EA..."
10,10 - 90,5200.0,1600.0,220.0,-30 - 70,220.4,-20 - 50,55.1,2.0,White,"IEEE 802.11a,IEEE 802.11ac,IEEE 802.11b,IEEE 8...","100,1000","802.1x RADIUS,AES,EAP-FAST,EAP-PEAP,EAP-SIM,EA..."
12,10 - 90,1000.0,280.0,89.0,-30 - 70,140.0,0 - 40,31.5,3.0,White,"IEEE 802.11a,IEEE 802.11ac,IEEE 802.11b,IEEE 8...","10,100,1000","802.1x RADIUS,AES,EAP,EAP-FAST,EAP-PEAP,EAP-SI..."
14,10 - 90,1000.0,1040.0,221.0,-30 - 70,54.0,0 - 40,221.0,1.0,White,"IEEE 802.11a,IEEE 802.11b,IEEE 802.11d,IEEE 80...","10,100,1000","802.1x RADIUS,AES,EAP-FAST,EAP-PEAP,EAP-SIM,EA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2228,10 - 90,5200.0,2090.0,220.0,-30 - 70,220.0,0 - 40,62.5,2.0,White,"IEEE 802.11a,IEEE 802.11ac,IEEE 802.11b,IEEE 8...","100,1000,2500,5000","802.1x RADIUS,AES,EAP,EAP-FAST,EAP-PEAP,EAP-SI..."
2230,10 - 90,1000.0,1000.0,221.0,-30 - 70,221.0,0 - 40,51.0,3.0,White,"IEEE 802.11a,IEEE 802.11ac,IEEE 802.11b,IEEE 8...","10,100,1000","802.1x RADIUS,AES,EAP,EAP-FAST,EAP-PEAP,EAP-SI..."
2231,10 - 85,867.0,303.0,123.0,-20 - 70,123.0,0 - 40,31.0,1.0,White,"IEEE 802.11ac,IEEE 802.11b,IEEE 802.11e,IEEE 8...","10,100,1000","WPA,WPA2"
2233,10 - 90,1300.0,1130.0,221.0,-30 - 70,221.0,0 - 40,54.0,2.0,White,"IEEE 802.11a,IEEE 802.11ac,IEEE 802.11d,IEEE 8...","10,100,1000","802.1x RADIUS,AES,EAP-FAST,EAP-PEAP,EAP-SIM,EA..."


In [130]:
pruned_data["Ethernet LAN data rates.3768"]

6              10,100,1000
7                 100,1000
10                100,1000
12             10,100,1000
14             10,100,1000
               ...        
2228    100,1000,2500,5000
2230           10,100,1000
2231           10,100,1000
2233           10,100,1000
2234           10,100,1000
Name: Ethernet LAN data rates.3768, Length: 1316, dtype: object

In [125]:
pruned_data["Ethernet LAN data rates.3768"].unique()

array(['10,100,1000', '100,1000', '100,1000,2500,5000', '10,100',
       '10,100,1000,2500,5000', '1000', '10,100,1000,2500', '10',
       '2500,5000,100,1000', '10,1000', '100,1000,2500', '10,1000,100',
       '10,100,10000', '100,1000,5000', '100', '1000,2500'], dtype=object)

In [ ]:
xgdata_mice_forest_amp["WPA2"][idx_nan]

In [ ]:
xgdata_mice_forest.head()

In [ ]:
1 == 1.0

# OLD STUFF!!!!!

In [ ]:
category_data_xgboost = category_data[["name", "Height.1464", "Width.1649", "Depth.1650", "Maximum data transfer rate.1165", "Product colour.1766"]]
category_data_xgboost = category_data_xgboost[~(category_data_xgboost.isin(["None", "nan", "NaN"]).any(axis=1) | category_data_xgboost.isnull().any(axis=1))].reset_index(drop=True)

category_data_xgboost_prod_col_rm = remove_data(category_data_xgboost, 0.3, ["Product colour.1766"])
category_data_xgboost_col_nan_idx = category_data_xgboost_prod_col_rm[(category_data_xgboost_prod_col_rm["Product colour.1766"].isnull()) | (category_data_xgboost_prod_col_rm["Product colour.1766"] == "None")].index
category_data_xgboost_train = category_data_xgboost_prod_col_rm.drop(category_data_xgboost_col_nan_idx, axis=0)
category_data_xgboost_test = category_data_xgboost_prod_col_rm[category_data_xgboost_prod_col_rm.index.isin(category_data_xgboost_col_nan_idx)]

#specific to this use case
cols = ["Height.1464", "Width.1649", "Depth.1650", "Maximum data transfer rate.1165"]
category_data_xgboost_train[cols] = category_data_xgboost_train[cols].astype(float)
category_data_xgboost_test[cols] = category_data_xgboost_test[cols].astype(float)
###


display(category_data_xgboost_train)
display(category_data_xgboost_test)

## Working XGBoost

In [ ]:
category_data_xgboost.groupby("Product colour.1766").count()

In [ ]:
input_data = category_data_xgboost[~category_data_xgboost["Product colour.1766"].isin(["Grey,White",  "Gold,White"])] # Got rid of 2 colors as they only showed up once
float_cols = ["Height.1464", "Width.1649", "Depth.1650", "Maximum data transfer rate.1165"]
int_cols = ["Product colour.1766"]
input_data[float_cols] = input_data[float_cols].astype(float)

y_mapping = {v:int(k) for k, v in enumerate(input_data["Product colour.1766"].unique())}

X_cols = input_data.drop(columns=["name", "Product colour.1766"])
y_cols = input_data["Product colour.1766"].map(y_mapping)

X_train, X_test, y_train, y_test = train_test_split(X_cols, y_cols, random_state=42, stratify=y_cols)

In [ ]:
display(pd.concat([X_train, y_train], axis=1))
display(pd.concat([X_test, y_test], axis=1))


In [ ]:
clf_xgb = xgb.XGBClassifier(objective="reg:logistic", seed=42)
clf = clf_xgb.fit(X_train,
            y_train)

## Sense of accuracy

In [ ]:
predictions = clf.predict(X_test)
cm = confusion_matrix(y_test, predictions, labels=[0, 1, 2, 3, 5])
cm

In [ ]:
y_mapping
# list(range(7))

In [ ]:
print(f'White accuracy: {361/(361+11+1):.1%}')
print(f'Grey accuracy: {56/(56+5):.1%}')
print(f'Black accuracy: {7/(7+2):.1%}')
print(f'Silver accuracy: {0/1:.1%}')
print(f'Bronze,Whilte: {1/1:.1%}')

In [ ]:
cross_val_score(clf, X_cols, y_cols, cv=2)

In [ ]:
cols = ["Height.1464", "Width.1649", "Depth.1650", "Maximum data transfer rate.1165"]
category_data_xgboost_train[cols] = category_data_xgboost_train[cols].astype(float)
category_data_xgboost_test[cols] = category_data_xgboost_test[cols].astype(float)

Sources:

https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/

https://medium.com/swlh/impute-missing-values-the-right-way-c63735fccccd

In [ ]:
category_data_xgboost_train.groupby(["name"]).filter(lambda x: len(x) > 5)

In [ ]:
tst = category_data[category_data["name"] == "Aironet 1572EAC"]
tst.drop(columns=["index", "id"]).drop_duplicates().to_csv('output.csv', index=False)


In [ ]:
category_data_xgboost_train = pd.get_dummies(category_data_xgboost_train.drop(columns=["name", "Maximum data transfer rate.1165"])) \
                                    .join(other=category_data_xgboost_train[["name", "Maximum data transfer rate.1165"]])

model = XGBRegressor()
model.fit(train_factorized.drop("Product colour.1766", axis=1), train_factorized["Product colour.1766"])
pred_col = model.predict(test_factorized.drop("Product colour.1766", axis=1))

In [ ]:
def factorize(df):
    df_copy = df.copy()
    for col in df_copy:
        unique_items = df[col].dropna().unique().tolist()
        mapping_dict = {k:v for v, k in enumerate(unique_items)}
        df_copy[col] = df_copy[col].map(mapping_dict)
    return df_copy

train_factorized = factorize(category_data_xgboost_train)
test_factorized = factorize(category_data_xgboost_test)

In [ ]:
test_factorized["name"][4]

In [ ]:
model = XGBRegressor()
model.fit(train_factorized.drop("Product colour.1766", axis=1), train_factorized["Product colour.1766"])
pred_col = model.predict(test_factorized.drop("Product colour.1766", axis=1))
pred_col.astype(int)

unique_items = category_data_xgboost_train["Product colour.1766"].dropna().unique().tolist()
mapping_dict = {k:v for v, k in enumerate(unique_items)}
encoding_map = {v:k for k,v in mapping_dict.items()}

category_data_xgboost_prod_col_rm_copy = category_data_xgboost_prod_col_rm.copy()
category_data_xgboost_prod_col_rm_copy["Product colour.1766"][category_data_xgboost_col_nan_idx] = pd.Series(pred_col.astype(int)).map(encoding_map)


In [ ]:
display(category_data_xgboost_prod_col_rm_copy[category_data_xgboost_prod_col_rm_copy.index.isin(category_data_xgboost_col_nan_idx)])
display(category_data_xgboost[category_data_xgboost.index.isin(category_data_xgboost_col_nan_idx)])

merged = category_data_xgboost_prod_col_rm_copy[category_data_xgboost_prod_col_rm_copy.index.isin(category_data_xgboost_col_nan_idx)].join(category_data_xgboost[category_data_xgboost.index.isin(category_data_xgboost_col_nan_idx)], how='inner', rsuffix='t2')
merged["acc"] = merged["Product colour.1766"] == merged["Product colour.1766t2"]
merged["acc"] = merged["acc"].apply(lambda x: int(x))
merged["acc"].sum()

In [ ]:
tree = ET.parse('IceCat Specifications/CategoriesList.xml')
root = tree.getroot()

In [ ]:
# Get all categories from specification
category_parent_list = []
for category in root[0][0].findall('Category'):
    cat_id = category.attrib
    parent_id = category.find('ParentCategory').attrib
    category_parent_list.append((int(cat_id["ID"]), int(parent_id["ID"])))

# Construct tree of categories from specification
all_nodes = {n[0]: {} for n in category_parent_list}
tree = {}
for item in category_parent_list:
    id, parent = item
    if parent != 1:
        all_nodes[parent][id] = all_nodes[id]
    else:
        tree[id] = all_nodes[id]




In [ ]:
# Get categories from provided feather files
feather_file_names = os.listdir("IceCat_Cat_2833_feather")
feather_categories = [int(re.search(r'[0-9]+', file_name).group(0)) for file_name in feather_file_names]


def dict_key_filter(obj, obj_filter):
    '''
    Filters dictionary to only include `obj` keys that are included in `obj_filter`. 
    https://stackoverflow.com/questions/31710271/how-to-filter-by-keys-through-a-nested-dictionary-in-a-pythonic-way
    '''
    def inner_dict_key_filter(obj): return dict_key_filter(obj, obj_filter)
    def to_keep(subtree): return not isinstance(subtree, (dict, list)) or subtree

    def build_subtree(key, value):
        if key in obj_filter:
            return copy.deepcopy(value) # keep the branch
        elif isinstance(value, (dict, list)):
            return inner_dict_key_filter(value) # continue to search
        return [] # just an orphan value here

    if isinstance(obj, dict):
        key_subtree_pairs = ((key, build_subtree(key, value)) for key, value in obj.items())
        return {key:subtree for key, subtree in key_subtree_pairs if to_keep(subtree)}
    elif isinstance(obj, list):
        return list(filter(to_keep, map(inner_dict_key_filter, obj)))
    return []


filtered_tree = dict_key_filter(tree, feather_categories)




In [ ]:
filtered_tree

In [ ]:

def flatten(d):    
    res = []
    if isinstance(d, dict):
        for key, val in d.items():
            res.append(key)
            res.extend(flatten(val))
    return res

collapsed_cat_filtered_tree = flatten(filtered_tree)
len(collapsed_cat_filtered_tree)

In [ ]:
filtered_tree[2833][2][55][60]

In [ ]:
cpy = copy.copy(collapsed_cat_filtered_tree)
cpy.sort()
len(cpy)

In [ ]:
level_one_count = 0
level_two_count = 0
level_three_count = 0
level_four_count = 0
level_one_items  = []
level_two_items = []
level_three_items = []
level_four_items = []
for level_one in filtered_tree.keys():
    level_one_count += 1
    level_one_items.append(level_one)
    for level_two in filtered_tree[level_one].keys():
        level_two_count += 1
        level_two_items.append(level_two)
        for level_three in filtered_tree[level_one][level_two].keys():
            level_three_items.append(level_three)
            level_three_count += 1
            for level_four in filtered_tree[level_one][level_two][level_three].keys():
                level_four_count += 1
                level_four_items.append(level_four)



In [ ]:
[level_one_count, level_two_count, level_three_count, level_four_count]

In [ ]:
level_three_items

In [ ]:
def flatten(d):    
    res = []  # Result list
    if isinstance(d, dict):
        for key, val in d.items():
            res.append(key)
            res.extend(flatten(val))
    # elif isinstance(d, list):
    #     res = d        
    # else:
    #     raise TypeError("Undefined type for flatten: %s"%type(d))

    return res


dict1 = {
    'Bob': {
        'shepherd': [4, 6, 3],
        'collie': [23, 3, 45],
        'poodle': [2, 0, 6],
    },
    'Sarah': {
        'shepherd': [1, 2, 3],
        'collie': [3, 31, 4],
        'poodle': [21, 5, 6],
    },
    'Ann': {
        'shepherd': [4, 6, 3],
        'collie': [23, 3, 45],
        'poodle': [2, 10, 8],
    }
}

print( flatten(dict1) )

In [ ]:
level_two_items

In [ ]:
len(res[2833])


In [ ]:
st ='frame_IceCat_Category_1007.feather'
re.search(r'[0-9]+', st).group(0)